In [3]:
import os
import sys
from pyspark import SparkContext
import matplotlib.pyplot as plt
import numpy as np
from pyspark.sql import SparkSession
import ipywidgets as widgets
from IPython.display import clear_output 


%matplotlib inline
%pylab inline
matplotlib.rcParams['figure.figsize'] = (10, 6)

ReqData = "/Users/pushparajparab/Desktop/Hospital_Revised_Flatfiles/Payment_and_Value_of_Care-Hospital.csv"
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(ReqData)
df.registerTempTable('Data')

NationalRate = "/Users/pushparajparab/Desktop/Hospital_Revised_Flatfiles/Payment-National.csv"
df2 = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(NationalRate)

df2.registerTempTable('Master')


category= "SELECT Measure_Name from Master"
categoryName = spark.sql(category)


state= "SELECT distinct State from Data order by State"
stateName = spark.sql(state)

_dataList = categoryName.collect()
_dataListState = stateName.collect()
# print(_dataList)
_category =[]
_selectedCat = ["NA"]
_category.append("NA")
for i in range(len(_dataList)):
    _category.append(_dataList[i][0])

    
_state =[]
_selectedSta = ["All"]
_state.append("All")
for i in range(len(_dataListState)):
    _state.append(_dataListState[i][0])    
    
selectState = widgets.Dropdown(
    options=_state,
    description='State:',
    disabled=False
)   
display(selectState)
select = widgets.Dropdown(
    options=_category,
    description='Category:',
    disabled=False
)
display(select)
    

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output()
        _selectedCat[0] = (change['new'])    
 
def on_change_state(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output()
        _selectedSta[0] = (change['new'])   

select.observe(on_change)
selectState.observe(on_change_state)
clear_output()




# Measure : Payment for heart failure patients

In [4]:
if _selectedCat[0] == "Payment for heart failure patients":
   t= 15959
elif _selectedCat[0] == "Payment for heart attack patients":
   t = 22760 
else:
   t = 14817

# print(t)
# print(_selectedSta[0])

query = " SELECT Hospital_Name,Payment,Value_of_care_category,M.National_payment as th"
query += " ,(CASE WHEN Value_of_care_category LIKE 'Average mortality%' then " +str(t) 
query += " WHEN Value_of_care_category LIKE 'Better mortality%' then " +str(t + 1000) 
query += " WHEN Value_of_care_category LIKE 'Worse mortality%' then " +str(t - 1000) +" ELSE NULL END) AS Mor" 
query += " FROM Data D JOIN Master M"
query += " ON D.Payment_measure_ID = M.Measure_ID"
query += " WHERE Payment != 'Not Available'"


if _selectedSta[0] != "All":
    query += " AND State = '" +  _selectedSta[0] + "'" 
    

query += " AND Payment_measure_name = '" +  _selectedCat[0] + "'"
query += " ORDER by D.Hospital_Name"

# print(query)
distint = spark.sql(query)
#  distint.show();



_x=[]
_y=[]
_text=[]
_th = []
_Mor =[]
_dataList = distint.collect()
for i in range(len(_dataList)):
    _x.append(_dataList[i][0])
    _y.append(_dataList[i][1])
    _th.append(_dataList[i][3])
    _text.append(_dataList[i][2])
    _Mor.append(_dataList[i][4])
    
# print(_selectedCat[0])

import plotly
# Set Plotly credetials

plotly.tools.set_credentials_file(username='pparab', api_key='jgQSn7NrsQAUnpx26Aks')
import plotly.plotly as py
import plotly.tools as plotly_tools
import plotly.graph_objs as go
from plotly.graph_objs import *
 #Sign in the plotly API
py.sign_in("pparab", "jgQSn7NrsQAUnpx26Aks")

Payment = go.Scatter(
    x=_x,
    y=_y,
    name = 'Payment',
    text = _text
)

Mort = go.Scatter(
    x=_x,
    y=_Mor,
    name = 'Mortality',
    hoverinfo='none'
)

National_Rate = go.Scatter(
    x=_x,
    y=_th,
    name = 'National Rate'
)

data = [Payment,National_Rate,Mort]
py.iplot(data, filename='basic-line')